# Imports

Uncomment and run this block if you see an error `No module named "utils" found`.

In [ ]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
import glob
import os

from matplotlib import pyplot as plt
import cv2
import numpy as np
import torch
import torch.nn as nn

from aimet_torch.quantsim import QuantizationSimModel
from aimet_torch.qc_quantize_op import QuantScheme
from utils.imresize import imresize
import utils.blocks as blocks

# Global Constants

These are all the constant variables that we use throughout this notebook. It also specifies the different model variations that were trained. You can select the necessary model at the end of this notebook while running inference.

In [ ]:
RGB_WEIGHTS = torch.FloatTensor([65.481, 128.553, 24.966])

MODEL_ARGS = {
    'release_abpn_28_2x': {
        'in_channels': 3,
        'out_channels': 3,
        'num_channels': 28,
        'scaling_factor': 2,
    },
    'release_abpn_28_3x': {
        'in_channels': 3,
        'out_channels': 3,
        'num_channels': 28,
        'scaling_factor': 3,
    },
    'release_abpn_28_4x': {
        'in_channels': 3,
        'out_channels': 3,
        'num_channels': 28,
        'scaling_factor': 4,
    },
    'release_abpn_32_2x': {
        'in_channels': 3,
        'out_channels': 3,
        'num_channels': 32,
        'scaling_factor': 2,
    },
    'release_abpn_32_3x': {
        'in_channels': 3,
        'out_channels': 3,
        'num_channels': 32,
        'scaling_factor': 3,
    },
    'release_abpn_32_4x': {
        'in_channels': 3,
        'out_channels': 3,
        'num_channels': 32,
        'scaling_factor': 4,
    }
}

#NOTE: Set the following variable to the path where your checkpoint folders are located
# (parent directory of the model checkpoint folder)
CHECKPOINT_DIR = '<checkpoint_parent_folder>'
#NOTE: Set the following variable to the path of your dataset (parent directory of actual images)
DATA_DIR = '<root>/set5/SR_testing_datasets'

FILENAME_FLOAT32 = 'checkpoint_float32.pth.tar'  # full precision model
FILENAME_INT8 = 'checkpoint_int8.pth' # quantized model
ENCODINGS = 'checkpoint_int8.encodings' # encodings of the quantized models

MODELS = list(MODEL_ARGS.keys())

MODEL_NAME = 'ABPNRelease'
DATASET_NAME = 'Set14'
ENCODING_PATH = ''

# Model Definition

This is the model definition for the Anchor-based Plain Net from https://arxiv.org/abs/2105.09750.

In [ ]:
class ABPNRelease(nn.Module):
    """
    Anchor-based Plain Net Model implementation (https://arxiv.org/abs/2105.09750)

    2021 CVPR MAI SISR Winner -- Used quantization-aware training
    """

    def __init__(self,
                 in_channels=3,
                 out_channels=3,
                 num_channels=28,
                 scaling_factor=2):
        """
        :param in_channels:     number of channels for LR input (default 3 for RGB frames)
        :param out_channels:    number of channels for HR output (default 3 for RGB frames)
        :param num_channels:    number of feature channels for the convolutional layers (default 28 in paper)
        :param scaling_factor:  scaling factor for LR-to-HR upscaling (default 2 for 4x upsampling)
        """

        super().__init__()
        self.scaling_factor = scaling_factor

        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=num_channels, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=num_channels, out_channels=out_channels * scaling_factor ** 2,
                      kernel_size=(3, 3), padding=1)
        )

        self.anchor = blocks.AnchorOp(scaling_factor)  # (=channel-wise nearest upsampling)
        self.add_residual = blocks.AddOp()
        self.depth_to_space = nn.PixelShuffle(scaling_factor)

    def forward(self, input):
        residual = self.cnn(input)
        upsampled_input = self.anchor(input)
        output = self.add_residual(upsampled_input, residual)

        return self.depth_to_space(output)

# Helper methods

A bunch of functions to help us create the dataset, run model inference, as well as evaluate average PSNR for all test-set images.

In [ ]:
def load_dataset(test_images_dir, scaling_factor=2):
    """
    Load the images from the specified directory and develop the low-res and high-res images.
    
    :param test_images_dir:
        Directory to get the test images from
    :param scaling_factor:
        Scaling factor to use while generating low-res images from their high-res counterparts
    :return:
        Pre-processed input images for the model, and low-res and high-res images for visualization
    """
    # Input images for the model
    INPUTS_LR = []

    # Post-processed images for visualization
    IMAGES_LR = []
    IMAGES_HR = []

    # Load the test images
    for img_path in glob.glob(os.path.join(test_images_dir, '*')):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        lr_img, hr_img = preprocess(img, scaling_factor)
        
        INPUTS_LR.append(lr_img)
        IMAGES_LR.append(post_process(lr_img))
        IMAGES_HR.append(post_process(hr_img))

    return INPUTS_LR, IMAGES_LR, IMAGES_HR

In [ ]:
def create_hr_lr_pair(img, scaling_factor=2):
    """
    Create low-res images from high-res images.
    
    :param img:
        The high-res image from which the low-res image is created
    :param scaling_factor:
         Scaling factor to use while generating low-res images
    :return:
        low-res and high-res image-pair
    """
    height, width = img.shape[0:2]

    # Take the largest possible center-crop of it such that its dimensions are perfectly divisible by the scaling factor
    x_remainder = width % (scaling_factor)
    y_remainder = height % (scaling_factor)
    left = x_remainder // 2
    top = y_remainder // 2
    right = left + (width - x_remainder)
    bottom = top + (height - y_remainder)
    hr_img = img[top:bottom, left:right]

    hr_height, hr_width = hr_img.shape[0:2]

    hr_img = np.array(hr_img, dtype='float64')
    lr_img = imresize(hr_img, 1. / scaling_factor)  # equivalent to matlab's imresize
    lr_img = np.uint8(np.clip(lr_img, 0., 255.))  # this is to simulate matlab's imwrite operation
    hr_img = np.uint8(hr_img)

    lr_height, lr_width = lr_img.shape[0:2]

    # Sanity check
    assert hr_width == lr_width * scaling_factor and hr_height == lr_height * scaling_factor

    lr_img = convert_image(lr_img, source='array', target='[0, 1]')
    hr_img = convert_image(hr_img, source='array', target='[0, 1]')

    return lr_img, hr_img

In [ ]:
def convert_image(img, source, target):
    """
    Convert image from numpy-array to float-tensor for torch.
    
    :param img:
        Input image to perform conversion on
    :param source:
        The type of input image to be converted
    :param target:
        The type of image after conversion
    :return:
        The converted image from `source` to `target`
    """
    if source == 'array':
        img = torch.from_numpy(img.transpose((2, 0, 1))).contiguous()
        img = img.to(dtype=torch.float32).div(255)
    elif source == '[0, 1]':
        img = torch.clamp(img, 0, 1)  # useful to post-process output of models that can overspill
    
    if target == '[0, 1]':
        pass  # already in [0, 1]
    elif target == 'y-channel':
        # Based on definitions at https://github.com/xinntao/BasicSR/wiki/Color-conversion-in-SR
        # torch.dot() does not work the same way as numpy.dot()
        # So, use torch.matmul() to find the dot product between the last dimension of an 4-D tensor and a 1-D tensor
        img = torch.matmul(img.permute(0, 2, 3, 1), RGB_WEIGHTS.to(img.device)) + 16.
    
    return img

In [ ]:
def preprocess(img, scaling_factor=2):
    """
    Generate low-res images from high-res inputs, downsample, and convert to tensors.
    
    :param img:
        Input image to pre-process
    :param scaling_factor:
         Scaling factor to use while generating low-res and high-res image-pairs
    :return:
        Low-res and High-res image pairs after pre-processing
    """
    lr_img, hr_img = create_hr_lr_pair(img, scaling_factor)

    return lr_img, hr_img

In [ ]:
def post_process(img):
    """
    Undo all preprocessing steps to get the upsampled low-res and high-res images.
    
    :param img:
        The pre-processed image to be converted back for comparison
    :return:
        The image after reverting the changes done in the pre-processing steps
    """
    img = img.detach().cpu().numpy()
    img = np.clip(255. * img, 0., 255.)
    img = np.uint8(img)
    img = img.transpose(1, 2, 0)

    return img

In [ ]:
def imshow(image):
    """
    Helper method to plot an image using PyPlot.
    
    :param image:
        Image to plot
    """
    plt.imshow(image, interpolation='nearest')
    plt.tick_params(left=False,
                    bottom=False,
                    labelleft=False,
                    labelbottom=False)

In [ ]:
def compute_psnr(img_pred, img_true, data_range=255., eps=1e-8):
    """
    Compute PSNR between super-resolved and original images.
    
    :param img_pred:
        The super-resolved image obtained from the model
    :param img_true:
        The original high-res image
    :param data_range:
        Default = 255
    :param eps:
        Default = 1e-8
    :return:
        PSNR value
    """
    err = (img_pred - img_true) ** 2
    err = err.mean(dim=-1).mean(dim=-1)

    return 10. * torch.log10((data_range ** 2) / (err + eps))

In [ ]:
def evaluate_psnr(y_pred, y_true):
    """
    Evaluate individual PSNR metric for each super-res and actual high-res image-pair.
    
    :param y_pred:
        The super-resolved image from the model
    :param y_true:
        The original high-res image
    :return:
        The evaluated PSNR metric for the image-pair
    """
    y_pred = y_pred.transpose(2, 0, 1)[None] / 255.
    y_true = y_true.transpose(2, 0, 1)[None] / 255.

    sr_img = convert_image(torch.FloatTensor(y_pred),
                           source='[0, 1]',
                           target='y-channel')
    hr_img = convert_image(torch.FloatTensor(y_true),
                           source='[0, 1]',
                           target='y-channel')

    return compute_psnr(sr_img, hr_img)

In [ ]:
def evaluate_average_psnr(sr_images, hr_images):
    """
    Evaluate the avg PSNR metric for all test-set super-res and high-res images.
    
    :param sr_images:
        The list of super-resolved images obtained from the model for the given test-images
    :param hr_images:
        The list of original high-res test-images
    :return:
        Average PSNR metric for all super-resolved and high-res test-set image-pairs
    """
    psnr = []
    for sr_img, hr_img in zip(sr_images, hr_images):
        psnr.append(evaluate_psnr(sr_img, hr_img))

    average_psnr = np.mean(np.array(psnr))
    
    return average_psnr

In [ ]:
def pass_calibration_data(sim_model, calibration_data=None):
    """
    Helper method to compute encodings for the QuantizationSimModel object.
    
    :param sim_model:
        The QuantizationSimModel object to compute encodings for
    :param calibration_data:
        Tuple containing calibration images and a flag to use GPU or CPU
    """
    (images_hr, use_cuda) = calibration_data
    if use_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    sim_model.eval()
    
    with torch.no_grad():
        for img in images_hr:
            lr_img, hr_img = preprocess(img, sim_model.scaling_factor)
            input_img = lr_img.unsqueeze(0).to(device)
            sim_model(input_img)

# Model Inference

In [ ]:
def load_model(model_checkpoint, model_name, model_args, use_quant_sim_model=False, 
               encoding_path='', calibration_data=None, use_cuda=True):
    """
    Load model from checkpoint directory using the specified model arguments for the instance.
    Optionally, you can use the QuantizationSimModel object to load the quantized model.
    
    :param model_checkpoint:
        Path to model checkpoint to load the model weights from
    :param model_name:
        Name of the model as a string
    :param model_args:
        Set of arguments to use to create an instance of the model
    :param use_quant_sim_model:
        `True` if you want to use QuantizationSimModel, default: `False`
    :param encoding_path:
        Path to gather encodings for the quantized model
    :return:
        One of the FP32-model or the quantized model
    """
    if use_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    model = eval(model_name)(**model_args)
    print(f"Loading model from checkpoint : {model_checkpoint}")
    state_dict = torch.load(model_checkpoint, map_location='cpu')['state_dict']
    model.load_state_dict(state_dict)
    model.to(device)
    
    if use_quant_sim_model:
        # Specify input-shape based on current model specification
        dummy_input = torch.rand(1, 3, 512, 512).to(device)
        
        sim = QuantizationSimModel(model=model,
                                   dummy_input=dummy_input,
                                   quant_scheme=QuantScheme.post_training_tf_enhanced,
                                   default_output_bw=8, 
                                   default_param_bw=8)

        sim.set_and_freeze_param_encodings(encoding_path=encoding_path)

        sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                              forward_pass_callback_args=(calibration_data, use_cuda))

        return sim.model

    return model

In [ ]:
def run_model(model, inputs_lr, use_cuda):
    """
    Run inference on the model with the set of given input test-images.
    
    :param model:
        The model instance to infer from
    :param INPUTS_LR:
        The set of pre-processed input images to test
    :return:
        The super-resolved images obtained from the model for the given test-images
    """
    if use_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    model.eval()
    images_sr = []

    # Inference
    for count, img_lr in enumerate(inputs_lr):
        with torch.no_grad():
            sr_img = model(img_lr.unsqueeze(0).to(device)).squeeze(0)

        images_sr.append(post_process(sr_img))
    print('')

    return images_sr

# Visualize Results

In [ ]:
def visualize(images_lr, images_hr, images_sr):
    """
    Visualize test-images as low-res, super-res and high-res.
    
    :param images_lr:
        List of low-res images
    :param images_hr:
        List of high-res images
    :param images_sr:
        List of super-resolved images
    """
    num_images = len(images_lr)
    plt.figure(figsize=(16, 4 * num_images))

    count = 1
    for lr_img, hr_img, sr_img in zip(images_lr, images_hr, images_sr):
        # Sub-plot for Low-res images
        plt.subplot(num_images, 3, count)
        plt.title('LR')
        imshow(lr_img)
        count += 1

        # Sub-plot for High-res images
        plt.subplot(num_images, 3, count)
        plt.title('HR')
        imshow(hr_img)
        count += 1

        # Sub-plot for Super-res images
        plt.subplot(num_images, 3, count)
        plt.title('SR')
        imshow(sr_img)
        count += 1
    
    # Display results
    plt.show()

# Select a model

In [ ]:
MODELS

Select one of the models printed above by selecting the corresponding index in the cell below and set `use_quantized` to `True` if you want to test the quantized model, else `False`.

In [ ]:
index = 1  # model index
use_quantized = True  # Whether to use the INT8 quantized model or FP32 model
use_cuda = False  # Whether to use CUDA or CPU

print(f'{MODELS[index]} ({"int8" if use_quantized else "float32"}) will be used')

# Choose model
MODEL_CONFIG = MODELS[index]

# Path to desired model weights and encodings (if necessary)
if use_quantized:
    FILENAME = FILENAME_INT8
    ENCODING_PATH = os.path.join(CHECKPOINT_DIR, MODEL_CONFIG, ENCODINGS)
else:
    FILENAME = FILENAME_FLOAT32

# Path to model checkpoint and encodings (if necessary)
MODEL_PATH = os.path.join(CHECKPOINT_DIR, MODEL_CONFIG, FILENAME)

# Data loading

Load test-set images (low-res and high-res pairs)

In [ ]:
# Path to test images
TEST_IMAGES_DIR = os.path.join(DATA_DIR, DATASET_NAME)

# Get test images
INPUTS_LR, IMAGES_LR, IMAGES_HR = load_dataset(TEST_IMAGES_DIR, MODEL_ARGS.get(MODEL_CONFIG)['scaling_factor'])

# Create model instance and load weights

In [ ]:
# Load the model
model = load_model(MODEL_PATH, MODEL_NAME, MODEL_ARGS.get(MODEL_CONFIG), 
                   use_quant_sim_model=use_quantized, encoding_path=ENCODING_PATH, 
                   calibration_data=IMAGES_HR if use_quantized else None,
                   use_cuda=use_cuda)

# Model Inference

Run inference to get the respective super-resolved images

In [ ]:
# Run model inference on test images and get super-resolved images
IMAGES_SR = run_model(model, INPUTS_LR, use_cuda)

Calculate average-PSNR between the test-set high-res and super-resolved images

In [ ]:
# Get the average PSNR for all test-images
avg_psnr = evaluate_average_psnr(IMAGES_SR, IMAGES_HR)
print(f"\n--- Average PSNR : {avg_psnr:.4f} ---\n")

Visualize the LR, HR and SR images side-by-side

In [ ]:
# Visualize all test-images returned from the model
visualize(IMAGES_LR, IMAGES_HR, IMAGES_SR)